In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [2]:
from stockMarket.stockData import StockData

stock_data = StockData("S&P 500")

In [3]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(stock_data.companies)

### Perform Screening

In [16]:
import numpy as np
import pandas as pd

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.min, description="Min Net Income")

screener_list = [
    dividend_yield_screener,
    price_screener,
    trailing_pe_screener,
    forward_pe_screener,
    revenue_screener,
    operating_cashflow_screener,
    equity_ratio_screener_not_bank,
    equity_ratio_screener_bank,
    ebitda_margin_screener,
    net_income_screener
]

screened_contracts = screener.screen(screener_list)

### Collect Data

In [20]:
from stockMarket.core import contracts_to_df

df = contracts_to_df(screened_contracts)
df.to_csv("screened_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
df

,Name,Sector,Price,Trailing PE,Forward PE,Market Cap,Dividend Yield,Payout Ratio,Revenue,Revenue Growth,...,Net Income,Avg Net Income,Min Net Income,Max Net Income,EBIT Margin,EBITDA Margin,Operating Cash Flow,Equity Ratio,Earnings Date,Ex-Dividend Date
BAC,Bank of America Corporation,Financial Services,33.400,10.853896,10.579114,263945.224192,2.87,29.870000,98581.000,1.661393,...,26515.000,26582.000000,17894.000,31978.000,NaN,NaN,NaN,9.171615,2024-04-16,NaT
CNP,"CenterPoint Energy, Inc.",Utilities,27.620,21.880953,17.018518,17402.845184,2.90,59.520000,9321.000,-0.609528,...,1057.000,786.833333,-773.000,1792.000,19.547259,33.365519,1810.000,26.051990,2024-02-20,2024-02-14
CFG,"Citizens Financial Group, Inc.",Financial Services,33.610,10.731629,8.232843,15666.979840,5.00,53.670000,8224.000,6.840731,...,1608.000,1761.500000,1057.000,2319.000,NaN,NaN,NaN,10.966643,2024-04-17,2024-01-30
CTRA,Coterra Energy Inc.,Energy,25.340,8.750865,11.495455,19022.936064,3.16,57.090000,9051.000,82.606309,...,4065.000,1127.005833,100.393,4065.000,57.861010,75.825876,5456.000,62.865932,2024-02-20,NaT
CSX,CSX Corporation,Industrials,35.070,19.129728,16.086363,69935.243264,1.24,23.780000,14853.000,6.039621,...,4166.000,3803.833333,2765.000,5471.000,40.550730,50.649700,5619.000,30.098778,2024-04-18,NaT
EQT,EQT Corporation,Energy,35.970,4.915069,13.094891,14758.592512,1.76,8.220001,7497.689,28.512717,...,1770.965,-381.385833,-2244.568,1770.965,34.383501,56.603175,3465.560,49.283240,2024-02-13,NaT
EXC,Exelon Corporation,Utilities,34.905,16.283720,14.526970,34850.246656,4.11,65.930000,19078.000,-8.629835,...,2170.000,2426.500000,1706.000,3779.000,17.376035,35.894748,4870.000,25.950980,2024-02-21,NaT
FITB,Fifth Third Bancorp,Financial Services,35.020,10.826087,10.017242,23744.018432,3.90,42.240003,8375.000,3.853603,...,2446.000,2254.666667,1427.000,2770.000,NaN,NaN,6428.000,8.352294,2024-04-19,NaT
BEN,"Franklin Resources, Inc.",Financial Services,27.520,16.040697,9.546713,13671.589888,4.49,69.769996,7849.400,5.302281,...,882.800,1127.483333,764.400,1831.200,14.043112,19.717431,1138.700,39.563165,2024-01-29,NaT
GEN,Gen Digital Inc.,Technology,23.860,10.659193,10.425439,15229.796352,2.10,22.420000,3338.000,6.088316,...,1349.000,1299.166667,31.000,3887.000,36.488916,46.345117,757.000,13.795698,2024-01-31,NaT


### Write Email

In [69]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Screener"
body = screener.screening_details
attachment = "screened_contracts.csv"
write_email(emails, subject, body, attachment)